<a href="https://colab.research.google.com/github/PeacemakerDott/BigQueryAPI/blob/main/BigQueryAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Подключение к публичным данным

### 1.1 Производим импорт необходимых библиотек:

In [ ]:
!pip install pandasql
from pandasql import sqldf, load_meat, load_births
pysqldf = lambda q: sqldf(q, globals())
import numpy as np
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Аутентификация прошла успешно!')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Аутентификация прошла успешно!


In [ ]:
# Решил добавить чуть-чуть визуализации в проект
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

### 1.2 Создаем проект в Google BigQuery:




В Google Cloud в разделе `Manage Resources` создаем проект и даем ему название.

![](https://i.ibb.co/s1SzmNV/image.png)

Название моего проекта и идентификатор представлены на скриншоте. 

![](https://i.ibb.co/pwyDw9R/image.png)

Название - `BigQuery API`, идентификатор - `bigquery-api-375613`

### 1.3 Подключаемся к нему через API и для проверки посмотрим часть данных из публичного датасета.

In [ ]:
project_id = 'bigquery-api-375613' 

test_query = '''
  SELECT * FROM `august-monument-187809.Assignments.view_hits_for_sessions`
'''

test_df = pd.io.gbq.read_gbq(test_query, project_id=project_id, dialect='standard')

In [ ]:
test_df.head(1)

,hitId,anonymousId,hitTime,utm_campaign,utm_source,utm_medium
0,0a0a29e0-0c2d-11e8-b6de-ffc22b5b4218,0129fa30-0c2d-11e8-8bc5-d9061171bfdd,2018-02-07 17:33:42.265000+00:00,None,None,None


# 2. Пробные задачи

## Задача №1
Написать SQL запрос, который сконвертирует хиты в сессии, а затем посчитать количество сессий в результирующей таблице.

Датасет для дальнейших экспериментов - `august-monument-187809.Assignments.view_hits_for_sessions`.

Алгоритм. 
1. Убираем дубликаты
> В исходных данных есть значения с одним и тем же `hitId` и `anonymousId`, но разным временем хита. Пронумеруем такие значения через `ROW_NUMBER ()` и оставим только те, счетчик у которых равен 0. 
2. Через `LAG ()` создаем отдельный столбец, в котором сохраняем информацию по предыдущему хиту по времени.
3. Вычитаем из актуального времени хита время предыдущего хита, которое мы добавили в отдельный столбец на предыдущем пункте. Если результат получается больше или равным 1800, то значит между ближайшими хитами прошло 30 минут и можно считать, что началась новая сессия.
4. Через `SUM ()` складываем счетчики начала сессии, равные единице. Сумма по `anonymousId` в итоге покажет информацию о количестве сессий по пользователям.
5. Склеиваем через нижнее подчеркивание `anonymousId` и номер сессии по пользователю, который мы получили на предыдущем пункте.


In [ ]:
project_id = 'bigquery-api-375613' 


query_to_session = '''
  SELECT hitId as First_Hit_In_Session, anonymousId as ClientID, utm_campaign, utm_source, utm_medium, SessionID FROM (
    SELECT *, CONCAT(anonymousId, '_', Visit_Number) as SessionID FROM
      (
        SELECT *, SUM(First_Hit_Session) OVER (PARTITION BY anonymousId ORDER BY hitTime) as Visit_Number
          FROM
            (SELECT *, 
              CASE 
                WHEN TIMESTAMP_DIFF(hitTime,previous_hit, SECOND) IS NULL OR TIMESTAMP_DIFF(hitTime,previous_hit, SECOND) >= 1800 
                  THEN 1
                ELSE 0
                END as First_Hit_Session FROM
                  (
                  SELECT * EXCEPT (Duplicate_Is_True), LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime) as previous_hit FROM 
                    (
                      SELECT * FROM
                        (SELECT *, ROW_NUMBER () OVER (PARTITION BY hitid, anonymousid ORDER BY hitTime) - 1  as Duplicate_Is_True
                        FROM `august-monument-187809.Assignments.view_hits_for_sessions`)
                        WHERE Duplicate_Is_True = 0
                    )
                  )
            )
      )    
  )
  WHERE First_Hit_Session = 1;
'''
query_session_count = '''
CREATE OR REPLACE TEMP TABLE result_table AS
  SELECT hitId as First_Hit_In_Session, anonymousId as ClientID, utm_campaign, utm_source, utm_medium, SessionID FROM (
    SELECT *, CONCAT(anonymousId, '_', Visit_Number) as SessionID FROM
      (
        SELECT *, SUM(First_Hit_Session) OVER (PARTITION BY anonymousId ORDER BY hitTime) as Visit_Number
          FROM
            (SELECT *, 
              CASE 
                WHEN TIMESTAMP_DIFF(hitTime,previous_hit, SECOND) IS NULL OR TIMESTAMP_DIFF(hitTime,previous_hit, SECOND) >= 1800 
                  THEN 1
                ELSE 0
                END as First_Hit_Session FROM
                  (
                  SELECT * EXCEPT (Duplicate_Is_True), LAG(hitTime) OVER (PARTITION BY anonymousId ORDER BY hitTime) as previous_hit FROM 
                    (
                      SELECT * FROM
                        (SELECT *, ROW_NUMBER () OVER (PARTITION BY hitid, anonymousid ORDER BY hitTime) - 1  as Duplicate_Is_True
                        FROM `august-monument-187809.Assignments.view_hits_for_sessions`)
                        WHERE Duplicate_Is_True = 0
                    )
                  )
            )
      )    
  )
  WHERE First_Hit_Session = 1;
  
SELECT COUNT(SessionID) as Session_Counts FROM result_table;
'''

df_session = pd.io.gbq.read_gbq(query_to_session, project_id=project_id, dialect='standard')
df_session_count = pd.io.gbq.read_gbq(query_session_count, project_id = project_id, dialect = 'standard')

В результате двух запросов на SQL мы сформировали два датафрейма, в которых содержится сводная информация по сессиям `df_session` и также информация о количестве сессий в сумме за указанный промежуток времени `df_session_count`.   

Пояснение к результирующим датафреймам (таблицам) ниже: 


1.   Столбцы в таблице `df_session`:

*   `First_Hit_In_Session` - это идентификатор первого хита в сессии

*   `ClientID` - пользовательский идентификатор

*   `utm_campaign` - UTM метка кампании

*   `utm_source` - UTM метка источника

*   `utm_medium` - UTM метка канала трафика

*   `SessionID` - идентификатор сессии

2.   Столбцы в таблице `df_session_count`:

*   `Session_Counts` - количество сессий пользователей за выбранный промежуток времени





In [ ]:
display(df_session)
display(df_session_count)
print('\nОтвет: Количество итоговых сессий -', df_session_count.iloc[0,0])

,First_Hit_In_Session,ClientID,utm_campaign,utm_source,utm_medium,SessionID
0,61955f80-0c01-11e8-918c-dfb5fee8a2f6,61496260-0c01-11e8-918c-dfb5fee8a2f6,None,None,None,61496260-0c01-11e8-918c-dfb5fee8a2f6_1
1,bc92f330-0c0f-11e8-81ee-933f9dd449ee,bc908230-0c0f-11e8-81ee-933f9dd449ee,Hu6FE/orzevU8B8hTe3Gdw==,7Iv1Fvr6UZJ+cSM+GOglAw==,cpc,bc908230-0c0f-11e8-81ee-933f9dd449ee_1
2,93ca6030-0c2f-11e8-87ee-4d53188ce706,09f3c670-09e2-11e8-a2c6-0b00f7380134,None,yCLBtjhT7Sc7iWh6xQX5+g==,cpc,09f3c670-09e2-11e8-a2c6-0b00f7380134_1
3,b160ef30-0c5a-11e8-8b19-6d98a1e6cd9d,b15aada0-0c5a-11e8-8b19-6d98a1e6cd9d,None,None,None,b15aada0-0c5a-11e8-8b19-6d98a1e6cd9d_1
4,b662d510-0c1f-11e8-979c-5bfaf11af261,b65e8f50-0c1f-11e8-979c-5bfaf11af261,CJNiaxAkackGcR6j3+AO6w==,7Iv1Fvr6UZJ+cSM+GOglAw==,cpc,b65e8f50-0c1f-11e8-979c-5bfaf11af261_1
...,...,...,...,...,...,...
600,ef49a5a0-0bfe-11e8-9978-232777c17fad,ef4252a0-0bfe-11e8-9978-232777c17fad,None,None,None,ef4252a0-0bfe-11e8-9978-232777c17fad_1
601,b0425710-0c1e-11e8-92b7-657482f56419,b0229a10-0c1e-11e8-92b7-657482f56419,None,None,None,b0229a10-0c1e-11e8-92b7-657482f56419_1
602,a556fdd0-0c0d-11e8-9b21-0ffdfe97bc4a,4bf67270-0ab9-11e8-9b0c-8351ff0c4d86,None,None,None,4bf67270-0ab9-11e8-9b0c-8351ff0c4d86_1
603,83272d70-0c2b-11e8-9490-ad4034722cfe,831b1f80-0c2b-11e8-9490-ad4034722cfe,7CyejM5VDmcQeKKWFXPjJQ==,7Iv1Fvr6UZJ+cSM+GOglAw==,cpc,831b1f80-0c2b-11e8-9490-ad4034722cfe_1


,Session_Counts
0,605



Ответ: Количество итоговых сессий - 605


Я решил посмотреть немного информации справочной по сессиям. Какие каналы трафика встречаются чаще всего, а также какая рекламная кампания/источник оказались в топ 3 по количеству сессий.

In [ ]:
df_session_medium = df_session.groupby('utm_medium').count()['SessionID'].to_frame()
df_session_medium.rename({'SessionID':'Количество сессий'}, axis=1,inplace = True)
df_session_medium.sort_values(by='Количество сессий', ascending = False)

,Количество сессий
utm_medium,
cpc,173
email,21
cpv,2


Как видно из таблицы `df_session_medium` выше, самое большое количество откликов идет по каналу `cpc`, далее `email` и `cpv` соответственно.

In [ ]:
df_session_campaign = df_session.groupby('utm_campaign').count()['SessionID'].to_frame()
df_session_source = df_session.groupby('utm_source').count()['SessionID'].to_frame()
df_session_campaign.rename({'SessionID':'Количество сессий'}, axis=1,inplace = True)
df_session_source.rename({'SessionID':'Количество сессий'}, axis=1,inplace = True)
display(df_session_campaign.sort_values(by='Количество сессий', ascending = False).head(3))
display(df_session_source.sort_values(by='Количество сессий', ascending = False).head(3))

,Количество сессий
utm_campaign,
a9Yj/+1ZvP9IJ1OM0mHmfg==,30
Uiwkan+XEQuhr4f4nqG7yg==,17
CJNiaxAkackGcR6j3+AO6w==,16


,Количество сессий
utm_source,
7Iv1Fvr6UZJ+cSM+GOglAw==,120
yCLBtjhT7Sc7iWh6xQX5+g==,52
eUdUPXMqN2CImzRw+SVnXA==,21


In [ ]:
print('Количество уникальных пользователей, у которых было больше одной сессии:',df_session[~df_session["SessionID"].str.contains(r'_1$', regex=True, case=False)]['ClientID'].nunique())

Количество уникальных пользователей, у которых было больше одной сессии: 48


## Задача №2

Написать SQL запрос, который сформирует таблицу с уникальными идентификаторами пользователей, количеством их покупок и дохода по ним (отсортировать по убыванию дохода). Количество пользователей в сформированной таблице должно составлять 20% от общего дохода за июнь 2017 г.

Датасет для дальнейших экспериментов - `bigquery-public-data.google_analytics_sample.ga_sessions_2017*`.

Алгоритм
1.   Создаем через WITH AS две таблицы. В первой таблице `main_table` будет сводная информация по пользователям с их выручкой, количеством купленных товаров и номером транзакции. Во второй таблице `helper_table` будет информация о номере транзакции и счетчик с `ROW_NUMBER ()`
> Мне потребовалось добавить счетчик и вторую таблицу `helper_table`, так как в исходных данных в связи с раскрытием сессий через UNNEST появляется слишком много излишне похожих данных (дубликаты), такие данные мешают в конечном препроцессинге. 

  > Также не забываем про условия `eCommerceAction.action_type = '6'`
  и `p.productQuantity IS NOT NULL`. Первое условие даст выбрать только те хиты, где была совершена покупка, а второе условие уберет пустые строки с пустыми заказами. 

2.   Через `INNER JOIN` по `TransactionID` склеиваем две таблицы, указываем условие, что нам нужно отсеять только те значения, где номер у счетчика равен 1.

  > Для того, чтобы в дальнейшем можно было отсечь все значения больше 20 % выручки, нам нужно посчитать эту границу - `Total_Revenue_In_USD`. Также потребуется вычислить кумулятивную сумму по столбцу `Revenue_In_USD` для того, чтобы в дальнейшем через сравнение больше/меньше оставить только необходимых нам пользователей. 

3.   Через внешний запрос `SELECT ... FROM ((WITH ... AS ))` и условие `WHERE Cumul_Sum_Revenue < Total_Revenue_In_USD` организуем конечный результат.




In [ ]:
project_id = 'bigquery-api-375613' 


query = '''
SELECT ClientID, Revenue_in_USD, ProductQuantity FROM 
((WITH main_table as( 
  SELECT
    fullVisitorId as ClientID,totals.transactionRevenue/POWER(10, 6) AS Revenue_in_USD, SUM(p.productQuantity) as ProductQuantity, h.transaction.transactionId as TransactionID
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_2017*`,
    UNNEST(hits) as h, 
    UNNEST(h.product) as p
  WHERE
    _TABLE_SUFFIX BETWEEN '0701' AND '0731' 
  AND
  eCommerceAction.action_type = '6'
  AND p.productQuantity IS NOT NULL
  GROUP BY ClientID, Revenue_in_USD, TransactionID
  ORDER BY Revenue_in_USD DESC),
helper_table AS (
SELECT
    totals.transactionRevenue/POWER(10, 6) AS Revenue_in_USD, h.transaction.transactionId as transactionID, ROW_NUMBER () OVER (PARTITION BY h.transaction.transactionId) as Count_Number
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_2017*`,
    UNNEST(hits) as h, 
    UNNEST(h.product) as p
  WHERE
    _TABLE_SUFFIX BETWEEN '0701' AND '0731' 
  AND
  eCommerceAction.action_type = '6'
  AND p.productQuantity IS NOT NULL
  ORDER BY Revenue_in_USD DESC
)
SELECT main_table.ClientID, main_table.Revenue_in_USD, main_table.ProductQuantity, main_table.TransactionID, ROUND(SUM(helper_table.Revenue_in_USD) OVER (PARTITION BY Count_Number)*0.2,2) as Total_Revenue_In_USD, ROUND(SUM(helper_table.Revenue_in_USD) OVER (PARTITION BY Count_Number ORDER BY  main_table.Revenue_in_USD DESC , main_table.transactionID),2) as Cumul_Sum_Revenue, helper_table.Count_Number FROM main_table
INNER JOIN helper_table ON helper_table.transactionID = main_table.TransactionID
WHERE helper_table.Count_Number = 1
ORDER BY main_table.Revenue_in_USD DESC))
WHERE Cumul_Sum_Revenue < Total_Revenue_In_USD
'''

result_df = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')
result_df.head(25)

,ClientID,Revenue_in_USD,ProductQuantity
0,9417857471295131045,10589.14,2574
1,6236695646664370912,4009.56,464
2,9417857471295131045,2548.50,709
3,7445235885559107095,2490.00,1000
4,875410720669459903,2407.20,80
5,6052206455243513477,1794.40,226


Изначально мой итоговый запрос к этой задаче выглядел по-другому, не так как представлен в ячейке выше. Он создавал временную таблицу, помещал туда все необходимые данные для получения результата, но не обрезал всех клиентов, принесших 20 % выручки. Также в первоначальном варианте запроса я решил обьявить пару переменных, так как если бы, например, потребовалось поменять временное окно с месяца на весь летний сезон, то это было бы сделать проще и быстрее.

Описанный мной выше вариант более оптимальный с точки зрения трафика:
> Если потребуется применить различные агрегатные функции к временной таблице или, например, поменять 20 % на 25 %, то это сделается во много раз проще и потребует намного меньше денег для выполнения конечного запроса в BigQuery.

С другой стороны мне намного больше нравится визуально тот вариант запроса, который я в итоге и вставил в ячейку выше, так как все делается через один запрос, а не через два с разделением в виде `;`


Посмотрим визуально на корреляцию между выручкой и количеством товара. 

In [ ]:
result_df

,ClientID,Revenue_in_USD,ProductQuantity
0,9417857471295131045,10589.14,2574
1,6236695646664370912,4009.56,464
2,9417857471295131045,2548.50,709
3,7445235885559107095,2490.00,1000
4,875410720669459903,2407.20,80
5,6052206455243513477,1794.40,226


In [ ]:
fig = px.scatter(result_df.rename({'ProductQuantity':'Количество купленных товаров', 'Revenue_in_USD':'Выручка в $', 'ClientID':'ID клиента'},axis=1), x="Выручка в $", y="Количество купленных товаров", trendline="lowess")
fig.update_layout(
    font=dict(
        family="Courier New, monospace",
        size=15,  
        color="Black"
    )
)
fig.show()

Из графика выше видно, что справа сверху есть одно аномальное значение похожее на выброс.  Прямой корреляции между количеством покупок и выручкой явным образом не наблюдается в связи со скачкообразным видом графика. Возможно, это говорит о большом ценовом разбросе по магазину.

# 3. Приложение

Тот самый запрос, который я изначально и написал для третьей задачи.

In [ ]:
project_id = 'bigquery-api-375613' 


query = '''
DECLARE percentage INT64;
DECLARE start_date STRING;
DECLARE end_Date STRING;
SET start_date = '0701';
SET end_date = '0731';
SET percentage = 20;

CREATE OR REPLACE TEMP TABLE result_table AS

 WITH main_table as( 
  SELECT
    fullVisitorId as ClientID,totals.transactionRevenue/POWER(10, 6) AS Revenue_in_USD, SUM(p.productQuantity) as ProductQuantity, h.transaction.transactionId as TransactionID
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_2017*`,
    UNNEST(hits) as h, 
    UNNEST(h.product) as p
  WHERE
    _TABLE_SUFFIX BETWEEN start_date AND end_date 
  AND
  eCommerceAction.action_type = '6'
  AND p.productQuantity IS NOT NULL
  GROUP BY ClientID, Revenue_in_USD, TransactionID
  ORDER BY Revenue_in_USD DESC),
helper_table AS (
SELECT
    totals.transactionRevenue/POWER(10, 6) AS Revenue_in_USD, h.transaction.transactionId as transactionID, ROW_NUMBER () OVER (PARTITION BY h.transaction.transactionId) as Count_Number
  FROM
    `bigquery-public-data.google_analytics_sample.ga_sessions_2017*`,
    UNNEST(hits) as h, 
    UNNEST(h.product) as p
  WHERE
    _TABLE_SUFFIX BETWEEN start_date AND end_date 
  AND
  eCommerceAction.action_type = '6'
  AND p.productQuantity IS NOT NULL
  ORDER BY Revenue_in_USD DESC
)
SELECT main_table.ClientID, main_table.Revenue_in_USD, main_table.ProductQuantity, main_table.TransactionID, ROUND(SUM(helper_table.Revenue_in_USD) OVER (PARTITION BY Count_Number)*percentage/100,2) as Total_Revenue_In_USD, ROUND(SUM(helper_table.Revenue_in_USD) OVER (PARTITION BY Count_Number ORDER BY  main_table.Revenue_in_USD DESC , main_table.transactionID),2) as Cumul_Sum_Revenue, helper_table.Count_Number FROM main_table
INNER JOIN helper_table ON helper_table.transactionID = main_table.TransactionID
WHERE helper_table.Count_Number = 1
ORDER BY main_table.Revenue_in_USD DESC;

SELECT ClientID, Revenue_in_USD, ProductQuantity FROM result_table 
WHERE Cumul_Sum_Revenue < Total_Revenue_In_USD;
'''

result_df = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')
result_df.head(25)

,ClientID,Revenue_in_USD,ProductQuantity
0,9417857471295131045,10589.14,2574
1,6236695646664370912,4009.56,464
2,9417857471295131045,2548.50,709
3,7445235885559107095,2490.00,1000
4,875410720669459903,2407.20,80
5,6052206455243513477,1794.40,226
